In [2]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

experiment = Experiment(
  api_key="api key",
  project_name="project-name",
  workspace="workspace-name",
  log_graph=True, # Can be True or False.
  auto_metric_logging=True, # Can be True or False
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: The given API key api key is invalid on www.comet.com, please check it against the dashboard. Your experiment will not be logged 
For more details, please refer to: https://www.comet.com/docs/v2/api-and-sdk/python-sdk/warnings-errors/
COMET ERROR: The given API key api key is invalid on www.comet.com, please check it against the dashboard. Your experiment will not be logged 
For more details, please refer to: https://www.comet.com/docs/v2/api-and-sdk/python-sdk/warnings-errors/


In [3]:
experiment.set_name("HIP_distilled_roberta_bilstm-v1")

In [4]:
from transformers import TFRobertaModel, RobertaConfig, RobertaTokenizer
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import os
import random
import numpy
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import random
import sys
import math
from scipy import stats
from scipy.spatial import distance
import random
import os


BERT_MODEL = "roberta-base"  # Specify the RoBERTa model
MAX_LENGTH = 128
TEST_SIZE = 0.15
SEED = 42

USE_ATT = True

BERT_DROPOUT = 0.2
LSTM_UNITS = 512
DENSE_UNITS = 50
LSTM_DROPOUT = 0.1
DENSE_DROPOUT = 0.2
EPOCHS = 10 #(Default 10)
BATCH_SIZE = 32

def tokenize(sentences, tokenizer):
    input_ids, input_masks = [], []
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence,
                                       add_special_tokens=True,
                                       max_length=MAX_LENGTH,
                                       pad_to_max_length=True,
                                       return_attention_mask=True,
                                       return_token_type_ids=False)  # Roberta doesn't use token_type_ids
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])

    return (np.asarray(input_ids, dtype='int32'),
            np.asarray(input_masks, dtype='int32'))

    ## Define base Roberta configs
# config = RobertaConfig(dropout=BERT_DROPOUT,
#                        attention_dropout=BERT_DROPOUT,
#                        output_attentions=True)

config = RobertaConfig.from_pretrained(BERT_MODEL,
                                       output_hidden_states=False,
                                       attention_probs_dropout_prob =BERT_DROPOUT,
                                       output_attentions = True)
config.output_hidden_states = False
transformer_model = TFRobertaModel.from_pretrained(BERT_MODEL, config=config)

# Freeze the pre-trained layers for fine-tuning
for layer in transformer_model.layers[:3]:
    layer.trainable = False

# Defining tokenizer
tokenizer = RobertaTokenizer.from_pretrained(BERT_MODEL,
                                             add_special_tokens=True,
                                             max_length=MAX_LENGTH,
                                             pad_to_max_length=True)






C:\ProgramData\anaconda3\envs\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\ProgramData\anaconda3\envs\research\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification

In [5]:
def random_seed(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

random_seed(SEED)

In [6]:
# Roberta BiLSTM

In [7]:
# input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH, ),
#                                      name='input_token',
#                                      dtype='int32')
# input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH, ),
#                                        name='masked_token',
#                                        dtype='int32')
# embedding_layer = transformer_model(input_ids_in,
#                                     attention_mask=input_masks_in)[0]
# X = tf.keras.layers.Bidirectional(
#     tf.keras.layers.LSTM(LSTM_UNITS,
#                          return_sequences=True,
#                          dropout=LSTM_DROPOUT,
#                          recurrent_dropout=LSTM_DROPOUT,
#                          kernel_initializer='normal'))(embedding_layer)
# if USE_ATT:
#     X = tf.keras.layers.Attention(use_scale=True)([X, X])  # Use attention.
# X = tf.keras.layers.GlobalMaxPool1D()(X)
# X = tf.keras.layers.Dense(DENSE_UNITS,
#                           activation='relu',
#                           kernel_initializer='normal')(X)
# X = tf.keras.layers.Dropout(DENSE_DROPOUT)(X)
# X = tf.keras.layers.Dense(
#     1,
#     activation='linear',  # Can be with activation="sigmoid" here.
#     kernel_initializer='normal')(X)
# model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs=X)
# model.compile(
#     optimizer='adam',
#     loss='mean_squared_error',  # Treat HIP as a regression problem
#     metrics=['acc', tf.keras.metrics.RootMeanSquaredError()])
# model.summary()

In [8]:
# Roberta RCNN

In [9]:
# from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D

# # Create the model
# input_ids_in = Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
# input_masks_in = Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32')

# embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]

# # Bi-directional LSTM layer
# X_lstm = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(embedding_layer)

# # Convolutional layer
# X_conv = Conv1D(filters=128, kernel_size=2, activation='relu')(X_lstm)  # Apply CNN on LSTM output

# # Global Max Pooling layer
# X_pool = GlobalMaxPooling1D()(X_conv)

# # Dense layers
# X = Dense(DENSE_UNITS, activation='relu')(X_pool)
# X = Dropout(DENSE_DROPOUT)(X)
# X = Dense(1, activation='linear')(X)

# model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs=X)

# # Compile the model
# model.compile(optimizer='adam',
#               loss='mean_squared_error',
#               metrics=['accuracy', tf.keras.metrics.RootMeanSquaredError()])

# model.summary()


In [10]:
# Roberta RCNN with attention

In [11]:
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Attention, Concatenate
# Inputs
input_ids_in = Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
input_masks_in = Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32')

# Transformer Model for Embedding
embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]

# Bi-directional LSTM layer
X_lstm = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(embedding_layer)

# Convolutional layer
X_conv = Conv1D(filters=128, kernel_size=2, activation='relu')(X_lstm)  # Apply CNN on LSTM output

# Attention layer
USE_ATT = True  # set to True or False
if USE_ATT:
    X_att = Attention(use_scale=True)([X_conv, X_conv])
    # Combining original convolution outputs and attention outputs
    X_combined = Concatenate(axis=-1)([X_conv, X_att])
    X_pool = GlobalMaxPooling1D()(X_combined)
else:
    X_pool = GlobalMaxPooling1D()(X_conv)

# Dense layers
X = Dense(DENSE_UNITS, activation='relu')(X_pool)
X = Dropout(DENSE_DROPOUT)(X)
X = Dense(1, activation='linear')(X)  # regression output

# Final model
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs=X)

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy', tf.keras.metrics.RootMeanSquaredError()])

# Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_token[0][0]',            
 el)                            thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [12]:
import pandas as pd
input_data = pd.read_csv("../datasets/hate_norm_with_span.csv")

In [13]:


intensity_value = []
hate_sentences = []

for i in range(len(input_data)):
    intensity_value.append(int(input_data['Original_Intensity'][i]))
    hate_sentences.append(input_data['Sentence'][i])
    intensity_value.append(int(input_data['Normalized_Intensity'][i]))
    hate_sentences.append(input_data['Normalized_Sentence'][i])

c = list(zip(intensity_value, hate_sentences))
random.shuffle(c)
intensity_value, hate_sentences = zip(*c)

X_tr, X_te, y_tr, y_te = train_test_split(hate_sentences,
                                          intensity_value,
                                          test_size=TEST_SIZE,
                                          random_state=1)

train_input_ids, train_input_masks = tokenize(
    X_tr, tokenizer)
test_input_ids, test_input_masks = tokenize(
    X_te, tokenizer)
y_tr = np.asarray(y_tr)
y_te = np.asarray(y_te)

  0%|                                                                                         | 0/5145 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\ProgramData\anaconda3\envs\research\lib\site-packages\transformers\tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (

In [14]:
model.fit(x=[train_input_ids, train_input_masks],
          y=y_tr,
          epochs=10,
          validation_split=0.15,
          batch_size=BATCH_SIZE)




Epoch 1/10
137/137 [==============================] - 58s 338ms/step - loss: 3.8499 - accuracy: 0.0066 - root_mean_squared_error: 1.9621 - val_loss: 2.1923 - val_accuracy: 0.0052 - val_root_mean_squared_error: 1.4806
Epoch 2/10
137/137 [==============================] - 44s 318ms/step - loss: 2.5088 - accuracy: 0.0066 - root_mean_squared_error: 1.5839 - val_loss: 1.7367 - val_accuracy: 0.0052 - val_root_mean_squared_error: 1.3178
Epoch 3/10
137/137 [==============================] - 44s 318ms/step - loss: 2.0584 - accuracy: 0.0066 - root_mean_squared_error: 1.4347 - val_loss: 1.7278 - val_accuracy: 0.0052 - val_root_mean_squared_error: 1.3144
Epoch 4/10
137/137 [==============================] - 43s 317ms/step - loss: 1.8973 - accuracy: 0.0066 - root_mean_squared_error: 1.3774 - val_loss: 1.4427 - val_accuracy: 0.0052 - val_root_mean_squared_error: 1.2011
Epoch 5/10
137/137 [==============================] - 44s 318ms/step - loss: 1.7240 - accuracy: 0.0066 - root_mean_squared_error: 1.

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [16]:
print("TEST split", TEST_SIZE)
results = model.evaluate(x=[test_input_ids, test_input_masks], y=y_te)
print(results)
result = model.predict(x=[test_input_ids, test_input_masks])
result = np.array(result, dtype=np.float64)
result = result.flatten()
print("pear", stats.pearsonr(result, y_te))
print("cosine", 1 - distance.cosine(result, y_te))

TEST split 0.15
29/29 [==============================] - 7s 222ms/step - loss: 1.4654 - accuracy: 0.0066 - root_mean_squared_error: 1.2105
[1.4653772115707397, 0.0066006602719426155, 1.2105276584625244]
29/29 [==============================] - 6s 220ms/step
pear PearsonRResult(statistic=0.8114186820853257, pvalue=9.198447365035483e-214)
cosine 0.9802076570751925


In [17]:
experiment.end()

In [18]:
# Model architecture

In [19]:
tf.keras.utils.plot_model(
    model,
    to_file='model_rcnn_orig.png',
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
    show_trainable=False
)

TypeError: plot_model() got an unexpected keyword argument 'show_trainable'

In [ ]:
# Roberta BiLSTM CNN

In [ ]:
experiment.set_name("HIP_roberta_rcnn")

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Concatenate
from transformers import TFBertModel
import tensorflow as tf

# Set some parameters
MAX_LENGTH = 128
LSTM_UNITS = 512
DENSE_UNITS = 64
DENSE_DROPOUT = 0.2

# Load the BERT model
#transformer_model = TFBertModel.from_pretrained("bert-base-uncased")

# Create the model
input_ids_in = Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
input_masks_in = Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32')

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]

# Bi-directional LSTM layer
X_lstm = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(embedding_layer)

# Apply a pooling layer to the LSTM outputs
X_lstm = GlobalMaxPooling1D()(X_lstm)

# Convolutional layer
X_conv = Conv1D(filters=128, kernel_size=2, activation='relu')(embedding_layer)

# Global Max Pooling layer
X_pool = GlobalMaxPooling1D()(X_conv)

# Concatenation of all layers
X_concat = Concatenate(axis=1)([X_lstm, X_pool])

# Dense layers
X = Dense(DENSE_UNITS, activation='relu')(X_concat)
X = Dropout(DENSE_DROPOUT)(X)
X = Dense(1, activation='linear')(X)

model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs=X)

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy', tf.keras.metrics.RootMeanSquaredError()])

model.summary()


In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='model_rcnn.png',
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
    show_trainable=False
)

In [ ]:
model.fit(x=[train_input_ids, train_input_masks],
          y=y_tr,
          epochs=10,
          validation_split=0.1,
          batch_size=BATCH_SIZE)

print("TEST split", TEST_SIZE)
results = model.evaluate(x=[test_input_ids, test_input_masks], y=y_te)
print(results)
result = model.predict(x=[test_input_ids, test_input_masks])
result = np.array(result, dtype=np.float)
result = result.flatten()
print("pear", stats.pearsonr(result, y_te))
print("cosine", 1 - distance.cosine(result, y_te))

In [ ]:
experiment.end()